In [3]:
!pip install keras-tuner
!pip install mat73
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf
from tensorflow import keras
import numpy as np
import scipy.io as sio
from tensorflow.keras.utils import to_categorical
import mat73
from keras.callbacks import ReduceLROnPlateau
import sklearn.metrics
from sklearn.utils import shuffle 
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Conv3D, MaxPooling3D, BatchNormalization
import matplotlib.pyplot as plt

     |████████████████████████████████| 71kB 5.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=c568617cc6a595b5dfa5fe3cc60b93de551c7a7af1c44c6de8729b10856ca89d
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=fc80b5c4cb6fa4e1094abbb70af6bb70433b1a51f4c9387548b72c0803f822fd
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables
Mounted at /content/drive


In [12]:
mat_file = sio.loadmat('/content/drive/MyDrive/Corrected_Data/DE_data_all_corrected.mat')
mat_file_labels = sio.loadmat('/content/drive/MyDrive/Corrected_Data/label.mat')
count = 0
accuracy = []

data = mat_file['data']
labels = mat_file_labels['label']

labels = labels[0]
labels_edited = np.empty(675)
for i in range(0,45):
  labels_edited[i*15:(i+1)*15] = labels

data, labels_edited = shuffle(data, labels_edited)

max = np.max(data)
min = np.min(data)

#Normalizing data
data = data/max
data = (data - np.mean(data))/np.std(data)

#Splitting Dataset into train, validation, test 
train_labels = labels_edited[0:550]
test_labels = labels_edited[550:600]
train_data = data[0:550]
test_data = data[550:600]
final_test = data[600:675]
final_labels = labels_edited[600:675]
cf_labels = np.where(final_labels== -1, 2, final_labels) #Label in the form for confusion matrix
un, co = np.unique(cf_labels, return_counts=True)
print(f'Unique: {un}, Counts: {co}')

train_labels_reshaped = train_labels.reshape(-1,1) #Formatting for input to the CNN model
test_labels_reshaped = test_labels.reshape(-1,1)
final_labels_reshaped = final_labels.reshape(-1,1)

train_labels_reshaped = to_categorical(train_labels_reshaped, 3) #One Hot Encoding
test_labels_reshaped = to_categorical(test_labels_reshaped, 3)
final_labels_reshaped = to_categorical(final_labels_reshaped, 3)

rnn_train = train_data.reshape(550, 62, -1) #Formatting for CNN input
rnn_test = test_data.reshape(50, 62, -1)
rnn_train = np.transpose(rnn_train, (0,2,1))
rnn_test = np.transpose(rnn_test, (0,2,1))

final_rnn_test = final_test.reshape(75, 62, -1)
final_rnn_test = np.transpose(final_rnn_test, (0,2,1))

Unique: [0. 1. 2.], Counts: [31 26 18]


In [ ]:
#K Fold Cross Validation Test

from sklearn.model_selection import StratifiedKFold
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []; cvscores_rnn = []; cvscores_hybrid = []; cvscores_ensemble = []
for train, test in kfold.split(data, labels_edited):
  # CNN Model
  model = Sequential([
    Conv2D(filters= 64, kernel_size= 5, input_shape = (62,265,5), padding= 'same'), #kernel_regularizer=tf.keras.regularizers.l1(0.01), activity_regularizer=tf.keras.regularizers.l2(0.01)),
    Conv2D(filters= 64, kernel_size= 3, padding= 'same'),
    Conv2D(filters= 64, kernel_size= 3, padding= 'same'),
    MaxPooling2D(pool_size= 2,strides = 2),
    #BatchNormalization(),
    Dropout(rate= 0.3),
    Conv2D(filters= 128, kernel_size= 3, padding= 'same'), #kernel_regularizer=tf.keras.regularizers.l1(0.01), activity_regularizer=tf.keras.regularizers.l2(0.01)),
    Conv2D(filters= 128, kernel_size= 3, padding= 'same'),
    MaxPooling2D(pool_size= 2, strides = 2),
    #BatchNormalization(),
    Dropout(rate= 0.2),

    Conv2D(filters= 256, kernel_size= 3, padding= 'same'), #kernel_regularizer=tf.keras.regularizers.l1(0.01), activity_regularizer=tf.keras.regularizers.l2(0.01)),
    MaxPooling2D(pool_size= 2, strides = 2),
    #BatchNormalization(),
    Dropout(rate= 0.25),

    Conv2D(filters= 512, kernel_size= 3, padding= 'same'),
    MaxPooling2D(pool_size= 2, strides= 2),
    #BatchNormalization(),
    Dropout(rate= 0.3),
    
    Flatten(),

    Dense(512, activation= 'relu'), #kernel_regularizer=tf.keras.regularizers.l1(0.01), activity_regularizer=tf.keras.regularizers.l2(0.01)),
    #BatchNormalization(),
    Dropout(rate= 0.4),
    Dense(256, activation= 'relu'),
    #BatchNormalization(),
    Dropout(rate= 0.2),
    Dense(64, activation= 'relu'),
    #BatchNormalization(),
    Dense(3, activation= 'softmax')
  ])

  #Compile CNN Model
  model.compile(optimizer= keras.optimizers.Adam(learning_rate=9e-5), loss= keras.losses.categorical_crossentropy, metrics= ['accuracy', tf.keras.metrics.RootMeanSquaredError()])  
  
  #Fit the CNN model
  reduce_lr_cnn = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, min_lr=1e-7) #patience = 5 and factor = 0.9

  history = model.fit(
    train_data,
    train_labels_reshaped,
    batch_size = 64,
    epochs=100, 
    validation_data=(test_data, test_labels_reshaped),
    callbacks = [reduce_lr_cnn]
  )

  
  
  #LSTM model 
  model_rnn = Sequential([
    LSTM(units= 64, activation= 'tanh', input_shape= [1325, 62], return_sequences = True),
    Dropout(0.25),
    LSTM(units= 128, activation= 'tanh', return_sequences = True),
    Dropout(0.3),
    LSTM(units= 128, activation= 'tanh', return_sequences = True),
    Dropout(0.35),
    LSTM(units= 256, activation= 'tanh', return_sequences = True),
    Flatten(),

    Dense(256, activation= 'relu'),
    Dropout(0.25),
    Dense(128, activation= 'relu'),
    Dropout(0.35),
    Dense(64, activation= 'relu'),
    Dense(3, activation= 'softmax'),

  ])

  #Compile LSTM Model
  model_rnn.compile(optimizer= keras.optimizers.Adam(learning_rate=5e-5), loss= keras.losses.categorical_crossentropy, metrics= ['accuracy', tf.keras.metrics.RootMeanSquaredError()])
  
  #Fit LSTM Model
  reduce_lr_rnn = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=5, min_lr=1e-6) #patience = 5 and factor = 0.9

  history_rnn = model_rnn.fit(
    rnn_train,
    train_labels_reshaped,
    batch_size = 16,
    epochs=60,
    validation_data=(rnn_test, test_labels_reshaped),
    callbacks = [reduce_lr_rnn]
  )
  
  #Hybrid model 
  model_hybrid = Sequential([
    Conv2D(filters= 64, kernel_size= 5, input_shape = (62,265,5), padding= 'same'), 
    Conv2D(filters= 64, kernel_size= 3, padding= 'same'),
    Conv2D(filters= 64, kernel_size= 3, padding= 'same'),
    MaxPooling2D(pool_size= 2,strides = 2),
    #BatchNormalization(),
    Dropout(0.3),

    Conv2D(filters= 128, kernel_size= 3, padding= 'same'), 
    Conv2D(filters= 128, kernel_size= 3, padding= 'same'),
    MaxPooling2D(pool_size= 2, strides = 2),
    #BatchNormalization(),
    Dropout(0.35),
    
    Conv2D(filters= 256, kernel_size= 3, padding= 'same'), 
    #MaxPooling2D(pool_size= 2, strides = 2), # This wasn't here, all batch norm
    #BatchNormalization(),

    Reshape((66, 15*256), input_shape= (15, 66, 256)), #Important to reshape so data passed to LSTM Layer correctly
    
    LSTM(units= 64, activation= 'tanh', input_shape= [1325, 62], return_sequences = True),
    Dropout(0.25),
    LSTM(units= 128, activation= 'tanh', return_sequences = True),
    Dropout(0.3),
    LSTM(units= 128, activation= 'tanh', return_sequences = True),
    Dropout(0.35),
    LSTM(units= 256, activation= 'tanh', return_sequences = True),
    
    Flatten(),
    
    Dense(512, activation= 'relu'), 
    Dropout(0.25), 
    Dense(256, activation= 'relu'),
    Dropout(0.2),
    Dense(64, activation= 'relu'),
    Dense(3, activation= 'softmax')
  ])

  #Compile Hybrid Model 
  model_hybrid.compile(optimizer= keras.optimizers.Adam(learning_rate=1e-4), loss= keras.losses.categorical_crossentropy, metrics= ['accuracy', tf.keras.metrics.RootMeanSquaredError()])

  #Fit the Hybrid Model
  reduce_lr_hybrid = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, min_lr=1e-6) #patience = 5 and factor = 0.9

  history_hybrid = model_hybrid.fit(
    train_data,
    train_labels_reshaped,
    batch_size = 32,
    epochs=60,
    validation_data=(test_data, test_labels_reshaped),
    callbacks = [reduce_lr_hybrid]
  )
  
  #Evaluate and print on every run
  scores = model.evaluate(x= final_test, y = final_labels_reshaped)
  scores_rnn = model_rnn.evaluate(x= final_rnn_test, y = final_labels_reshaped)
  scores_hybrid = model_hybrid.evaluate(x= final_test, y = final_labels_reshaped)

  #Implementing Ensemble (Stacking method) using a meta model 

  #Generating Predictions
  pred = np.argmax(model.predict(train_data), axis=-1)
  pred_rnn = np.argmax(model_rnn.predict(rnn_train), axis=-1)
  pred_hybrid = np.argmax(model_hybrid.predict(train_data), axis=-1)
  inputs = [pred, pred_rnn, pred_hybrid]
  inputs = np.array(inputs).T

  pred_test = np.argmax(model.predict(test_data), axis=-1)
  pred_rnn_test = np.argmax(model_rnn.predict(rnn_test), axis=-1)
  pred_hybrid_test = np.argmax(model_hybrid.predict(test_data), axis=-1)
  inputs_test = [pred_test, pred_rnn_test, pred_hybrid_test]
  inputs_test = np.array(inputs_test).T

  #Meta Model
  model_stack = Sequential([
    Dense(128, 'relu', input_shape= (3,)),
    Dense(256, 'relu'),
    Dense(256, 'relu'),
    Dense(64, 'relu'),
    Dense(3, 'softmax')
  ])

  #Compiling the Meta Model
  model_stack.compile(optimizer= keras.optimizers.Adam(learning_rate=1e-4), loss= keras.losses.categorical_crossentropy, metrics= 'accuracy')

  #Fitting the Meta Model
  reduce_lr_stack = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, min_lr=1e-6) #patience = 5 and factor = 0.9

  history_stack = model_stack.fit(
    inputs,
    train_labels_reshaped,
    batch_size = 32,
    epochs=150,
    validation_data=(inputs_test, test_labels_reshaped),
    callbacks = [reduce_lr_stack]
  )

  #Evaluation, Print and store Results
  acc = model_stack.evaluate(x= final_preds.T, y= final_labels_reshaped)
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  print("%s: %.2f%%" % (model_rnn.metrics_names[1], scores_rnn[1]*100))
  print("%s: %.2f%%" % (model_hybrid.metrics_names[1], scores_hybrid[1]*100))
  cvscores.append(scores[1] * 100)
  cvscores_rnn.append(scores_rnn[1] * 100)
  cvscores_hybrid.append(scores_hybrid[1] * 100)
  cvscores_ensemble.append(acc[1])


  print("%s: %.2f%%" % (model_hybrid.metrics_names[1], scores_hybrid[1]*100))
  cvscores_hybrid.append(scores_hybrid[1] * 100)


In [ ]:
#Print results and Box and Whisker Plot for Hybrid model. Can be changed for each model.

fig, ax = plt.subplots()
ax.set_title('Hybrid K Fold Validation Test Boxplot (K=10)')
ax.boxplot((cvscores_hybrid), showfliers=True)
print(f'Accuracy Average: {np.mean(cvscores_hybrid)}, Standard Deviation: {np.std(cvscores_hybrid)}, Max: {np.max(cvscores_hybrid)}, Min: {np.min(cvscores_hybrid)}')

#Uncomment to save results and box and whisker plots
#np.savetxt('hybrid_accuracies.csv', cvscores_hybrid, delimiter=',')
#plt.savefig('Hybrid_boxplot.jpg')